In [ ]:
import requests
import csv
from datetime import datetime, timedelta

api_key = "29005d8e87b8a6f4f1379cbd7d0ccc77"

lat = 32.5027
lon = -117.00371

duration_per_request = 24 * 60 * 60 

num_days = 365

url_template = f"https://history.openweathermap.org/data/2.5/history/city?lat={lat}&lon={lon}&type=hour&start={{start_time}}&end={{end_time}}&appid={api_key}"

csv_filename = "Weather_Tijuana.csv"

with open(csv_filename, mode='w', newline='') as csv_file:
    fieldnames = ["timestamp", "temperature", "feels_like", "pressure", "humidity", "temp_min", "temp_max", "wind_speed", "wind_degree", "clouds", "weather_id", "weather_main", "weather_description", "weather_icon"]  # Modify fieldnames as needed
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

    
    writer.writeheader()

    for i in range(num_days):
        
        start_time = int((datetime.now() - timedelta(days=i+1)).timestamp())
        end_time = int((datetime.now() - timedelta(days=i)).timestamp())
        
        url = url_template.format(start_time=start_time, end_time=end_time)

        response = requests.get(url)

        if response.status_code == 200:
    
            weather_data = response.json()

            for hour_data in weather_data['list']:
                writer.writerow({
                    "timestamp": datetime.utcfromtimestamp(hour_data['dt']).strftime('%Y-%m-%d %H:%M:%S'),
                    "temperature": hour_data['main']['temp'],
                    "feels_like": hour_data['main']['feels_like'],
                    "pressure": hour_data['main']['pressure'],
                    "humidity": hour_data['main']['humidity'],
                    "temp_min": hour_data['main']['temp_min'],
                    "temp_max": hour_data['main']['temp_max'],
                    "wind_speed": hour_data['wind']['speed'],
                    "wind_degree": hour_data['wind']['deg'],
                    "clouds": hour_data['clouds']['all'],
                    "weather_id": hour_data['weather'][0]['id'],
                    "weather_main": hour_data['weather'][0]['main'],
                    "weather_description": hour_data['weather'][0]['description'],
                    "weather_icon": hour_data['weather'][0]['icon']
                    
                })

        else:
            print(f"Error for day {i+1}: {response.status_code} - {response.text}")

print(f"Data has been exported to {csv_filename}")